<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial: simple_ppm.c

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the functions `simple_ppm` and `simple_ppm_no_rho_P`. The function reconstructs $\rho_b$, $P$, and other selected primitives at the points $U_r = U(i-\frac{1}{2}+\epsilon)$ and $U_l = U(i-\frac{1}{2}-\epsilon)$. The second function does the same, but without reconstructing $\rho_b$ and $P$.

### Required and recommended citations:
* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#introduction): **Introduction**
1. [Step 2](#simple_ppm): **The `simple_ppm` function**
1. [Step 3](#ppm_Ur_Ul): **The `ppm_Ur_Ul` function**
    1. [Step 3.a](#slope_limit): **The `slope_limit` function**
    1. [Step 3.b](#compute_UrUl_onevar): **The `compute_UrUl_onevar` function**
    1. [Step 3.c](#steepen_rhor_rhol): **The `steepen_rhor_rhol` function**
    1. [Step 3.d](#shock_detection_ftilde): **The `shock_detection_ftilde` function**
    1. [Step 3.e](#flatten_and_monotonize_Ur_and_Ul): **The `flatten_and_monotonize_Ur_and_Ul` function**
1. [Step 4](#simple_ppm_no_rho_P): **The `simple_ppm_no_rho_P` function**
1. [Step 5](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial notebook we discuss the `simple_ppm` for reconstructing primitives onto faces. This is used to compute the fluxes and source terms for the evolution equations. It is direction-agnostic, so the same function can be used to reconstruct in any direction. This implementation reconstructs the left and right sides of the $i-\frac{1}{2}$ face using stencils of $\{i-3,i+1\}$ and $\{i-2,i+2\}$.

TODO: link to other ppm

<a id='simple_ppm'></a>

# Step 2: The `simple_ppm` function \[Back to [top](#toc)\]

$$\label{simple_ppm}$$


First, let's look at the arguments of `simple_ppm`:
```
void simple_ppm
      const double rho[6],
      const double pressure[6],
      const double var_data[][6],
      const int num_vars,
      const double v_flux_dirn[6],
      const double Gamma_eff,
      double *restrict rhor,
      double *restrict rhol,
      double *restrict pressr,
      double *restrict pressl,
      double *restrict var_datar,
      double *restrict var_datal)
```

The first three arguments are stencils for the variables to be reconstructed. The stencil length is 6 because each reconstruction requires 5 points, and we are reconstructing both the left and right faces. `var_data` is a 2D array containing stencils for all the other variables to be reconstructed. The first array argument will loop from 0 to `num_vars`, so the passed array should have dimensions `var_data[num_vars][6]`. `v_flux_dirn` is the velocity $v$ in the direction of the reconstruction. I.e., the quantity $v^x$ would be passed to reconstruct in the $x$ direction. Finally, `Gamma_eff` is the effective gamma for the reconstruction. This is computed in `IllinoisGRMHD` with
$$
  \Gamma_{th} + \frac{P_{\rm cold}}{P}(\Gamma_{\rm ppoly}[\rho] - \Gamma_{th})
$$

The remaining arguments are the outputs for the right and left values. `var_datar` and `var_datal` are arrays of length `num_vars`.

The actual `simple_ppm` function is quite short. It calls the `ppm_Ur_Ul` function twice, once for each point. The desired return values are
$$
U_r = U(i-\frac{1}{2}+\epsilon) \\
U_l = U(i-\frac{1}{2}-\epsilon)
$$
where $U$ represents the variable being reconstructed. first `ppm_Ur_Ul` call computes
$$
U^R(i) = U\left(i+\frac{1}{2}\right) \\
U^L(i) = U\left(i-\frac{1}{2}\right)
$$
using the stencil centered around $U(i)$. Thus, `simple_ppm` passes the last 5 points of the stencil to `ppm_Ur_Ul`. Since we are constructing the $i-\frac{1}{2}$ face, reconstructing from $i$ gives us the $+\epsilon$ side of the face:
$$
U_r = U(i-\frac{1}{2}+\epsilon) = U^{L}(i)
$$
Then, we pass the first 5 stencil points to `ppm_Ur_Ul`, which computes
$$
U^(i-1) = U\left(i-\frac{1}{2}\right) \\
U^L(i-1) = U\left(i-\frac{3}{2}\right)
$$
Reconstructing from $i-1$ gives us the $-\epsilon$ side of the face:
$$
U_l = U(i-\frac{1}{2}-\epsilon) = U^{R}(i-1)
$$

<a id='ppm_Ur_Ul'></a>

## Step 3: The `ppm_Ur_Ul` Function \[Back to [top](#toc)\]

$$\label{ppm_Ur_Ul}$$

The core of the reconstruction method is the `ppm_Ur_Ul` function. This function actually computes the right and left faces for a point. The arguments are nearly identical to `simple_ppm`
```
void ppm_Ur_Ul(
      const double rho[5],
      const double pressure[5],
      const double var_data[][5],
      const int num_vars,
      const double v_flux_dirn[5],
      const double Gamma_eff, // Gamma_eff = (partial P / partial rho0)_s /(P/rho0)
      double *restrict rhor,
      double *restrict rhol,
      double *restrict pressr,
      double *restrict pressl,
      double *restrict varsr,
      double *restrict varsl)
```
except that the stencils are 5 instead of 6. This stencil is centered on the point $i$ being reconstructed. The reconstruction process is split into several steps.

<a id='slope_limit'></a>

## Step 3.a: The `slope_limit` Function \[Back to [top](#toc)\]

$$\label{slope_limit}$$

The `slope_limit` function
```
double slope_limit(const double dU, const double dUp1)
```
returns the slope-limited $\Delta U$. This function is used by several functions inside the PPM routine. If we have a stencil for the variable $U$, then the call would take the form
$$
dU_i = \mathrm{slope\_limit}(U_{i} - U_{i-1}, U_{i+1} - U_i)
$$
This function is based on Eq. 60 in JOURNAL OF COMPUTATIONAL PHYSICS 123, 1-14 (1996) (note the factor of 2 missing in the $|a_{j+1} - a_{j}|$ term) TODO cite properly. Defining
$$
\Delta U_i = U_{i} - U_{i-1} \\
\Delta U_{i+1} = U_{i+1} - U_i
$$
`slope_limit` returns 0 if $dU_i$ and $dU_{i+1}$ have different signs. If not, it first computes the average
$$
\Delta U_{\rm avg} = \frac{1}{2}\left(\Delta U_i + \Delta U_{i+1}\right)
$$
and then sets the sign $\sigma$ of the slope using
$$
\sigma = (0.0 < \Delta U_{\rm avg}) - (\Delta U_{\rm avg} < 0.0)
$$
This ensures that $\sigma$ matches the signs of $\Delta U_i$ and $\Delta U_{i+1}$. Finally, the slope is determined by returning
$$
{\rm MIN} \left( \Delta U_{\rm avg}, 2U_i, 2U_{i+1} \right)
$$

<a id='compute_UrUl_onevar'></a>

## Step 3.b: The `compute_UrUl_onevar` Function \[Back to [top](#toc)\]

$$\label{compute_UrUl_onevar}$$

The `compute_UrUl_onevar` function computes the $U^R$ and $U^L$ values. It first computes the derivatives at $i-1$, $i$, and $i+1$ using the `slope_limit` function
\begin{align}
dU_{i-1} &= \mathrm{slope\_limit}(U_{i-1} - U_{i-2}, U_{i} - U_{i-1}) \\
dU_i     &= \mathrm{slope\_limit}(U_{i} - U_{i-1}, U_{i+1} - U_i) \\
dU_{i+1} &= \mathrm{slope\_limit}(U_{i+1} - U_{i}, U_{i+2} - U_{i+1})
\end{align}

Then,
$$
U^R = \frac{1}{2}\left(U_{i+1} + U_i\right) + \frac{1}{6}\left(dU_i - dU_{i+1}\right) \\
U^L = \frac{1}{2}\left(U_i + U_{i-1}\right) + \frac{1}{6}\left(dU_{i-1} - dU_i\right) \\
$$

<a id='steepen_rhor_rhol'></a>

## Step 3.c: The `steepen_rhor_rhol` Function \[Back to [top](#toc)\]

$$\label{steepen_rhor_rhol}$$

Reconstruction of $\rho_b$ gets special treatment

<a id='shock_detection_ftilde'></a>

## Step 3.d: The `shock_detection_ftilde` Function \[Back to [top](#toc)\]

$$\label{shock_detection_ftilde}$$

The `shock_detection_ftilde` function
```
double shock_detection_ftilde(const double P[5], const double v_flux_dirn[5]) {
```
returns $\tilde{f}$, which is used to flatten $U^R$ and $U^L$ in the presence of shocks. It requires the `pressure` and `v_flux_dirn` arguments which are passed to `ppm_Ur_Ul`. For this section, we refer to `v_flux_dirn` as $v$.

In addition to computing $\tilde{f}$, we also need to determine if there is a shock. To begin, we compute

\begin{align}
\Delta P_1 &= P_{i+1} - P_{i-1} \\
\Delta P_2 &= P_{i+2} - P_{i-2}
\end{align}

and

\begin{align}
{\rm avg}_1 &= \frac{1}{2} \left( P_{i-1} + P_{i+1} \right) \\
{\rm avg}_2 &= \frac{1}{2} \left( P_{i-2} + P_{i+2} \right)
\end{align}

We use these quantities determine whether or not there is a shock. If
$$
\frac{\Delta P_1}{{\rm avg}_1} < 1.5\times 10^{-15}
$$
then there is no shock. Otherwise, if the above is false and
$$
\frac{\Delta P_2}{{\rm avg}_2} < 1.5\times 10^{-15}
$$
is true, then there may be a shock. To find out, we compute

\begin{align}
q_1 &= 10\left( \frac{\Delta P_1}{\Delta P_2} -0.75 \right) \\
q_2 &= \frac{|\Delta P_1|}{{\rm MIN}(P_{i-1},P_{i+1})}
\end{align}

Using these, we can compute

$$
w_{\rm shock} = \left( q_2 > 0.33\ {\rm AND}\ q_2(v_{i-1} - v_{i+1}) > 0.0 \right)
$$
If $w_{\rm shock} == 1$, then we are inside a shock. Finally, we return
$$
{\rm MIN}\left( 1, w_{\rm shock} {\rm MAX}(0.0, q_1) \right)
$$
If we aren't in a shock ($w_{\rm shock} == 0$), then this returns 0. If we are in a shock ($w_{\rm shock} == 1$), then this returns the minimum of 1 and $q_1$, with the caveat that $q_1 > 0$. If $q_1 < 0$, it will always return 1.

<a id='flatten_and_monotonize_Ur_and_Ul'></a>

## Step 3.e: The `flatten_and_monotonize_Ur_and_Ul` Function \[Back to [top](#toc)\]

$$\label{flatten_and_monotonize_Ur_and_Ul}$$

The `flatten_and_monotonize_Ur_and_Ul` function
```
void flatten_and_monotonize_Ur_and_Ul(const double U, const double ftilde, double *restrict Ur, double *restrict Ul)
```
flattens the function in the case of shocks or steep gradients and then monotonizes the variables. For the variable $U$, we flatten the face values with
\begin{align}
U^R &= \tilde{f}U + U^R (1 - \tilde{f}) \\
U^L &= \tilde{f}U + U^L (1 - \tilde{f})
\end{align}

Then, we monotonize using the intermediate variables
\begin{align}
\Delta U &= U^R - U^L \\
U_{\rm tmp} &= \Delta U \left(U - \frac{1}{2}\left(U^R - U^L \right)\right)
\end{align}

There are three special cases we consider. First, if
$$
\left(U^R - U\right)\left(U - U^L\right) \le 0.0
$$
then
$$
U^R = U^L = U
$$

Second, if
$$
U_{\rm tmp} > \frac{1}{6}\left(\Delta U \right)^2
$$
then
$$
U^L = 3U - 2U^R
$$

Finally, if
$$
U_{\rm tmp} < -\frac{1}{6}\left(\Delta U \right)^2
$$
then
$$
U^R = 3U - 2U^L
$$

<a id='simple_ppm_no_rho_P'></a>

# Step 4: The `simple_ppm_no_rho_P` function \[Back to [top](#toc)\]

$$\label{simple_ppm_no_rho_P}$$

In addition to `simple_ppm`, we also provide the `simple_ppm_no_rho_P` function. This function is nearly identical to `simple_ppm`, but it does not reconstruct the baryonic density or pressure. We choose to provide two functions to avoid having many `if` statements within the functions. For example, the function `steepen_rhor_rhol` is only called if $\rho_b$ is being reconstructed, so including $\rho_b$ in the `var_data` array would require another argument to tell the code which index contained $\rho_b$ and a check on every reconstruction at every point whether the variable being reconstructed is $\rho_b$. Similarly, $P$ is required regardless of whether it is being reconstructed or not, as it is needed to compute $\tilde{f}$. We could add $P$ to `var_data` whenever we actually want to reconstruct it, but that would be duplicating the stencils. For these reasons, we provide two different functions.

The primary use for this function is reconstructing variables that will get overwritten during the `simple_ppm`. For example, reconstructing the velocities $\left(v^R\right)^L$ and $\left(v^R\right)^R$ can cause a race condition with the reconstruction of $v^R$ from $v$. If the $v^R$ computed from $v$ is copied into a temporary variable which serves as input for reconstructing $\left(v^R\right)^L$ and $\left(v^R\right)^R$, there would not be a conflict. However, that is a lot of duplicated memory, so many codes use $v^R$ as both output and input.

As always, let's first look at the arguments of `simple_ppm_no_rho_P`:
```
void ppm_Ur_Ul_no_rho_P(
      const double pressure[5],
      const double var_data[][5],
      const int num_vars,
      const double v_flux_dirn[5],
      const double Gamma_eff,
      double *restrict varsr,
      double *restrict varsl);

```
These arguments are nearly identical to `simple_ppm`. `pressure` contains the pressure stencil for computing $\tilde{f}$. `var_data` is a 2D array containing stencils for all the variables to be reconstructed. The first array argument will loop from 0 to `num_vars`, so the passed array should have dimensions `var_data[num_vars][6]`. The stencil length is 6 because each reconstruction requires 5 points, and we are reconstructing both the left and right faces. `v_flux_dirn` is the velocity $v$ in the direction of the reconstruction. I.e., the quantity $v^x$ would be passed to reconstruct in the $x$ direction. Finally, `Gamma_eff` is the effective gamma for the reconstruction. This is computed in `IllinoisGRMHD` with
$$
  \Gamma_{th} + \frac{P_{\rm cold}}{P}(\Gamma_{\rm ppoly}[\rho] - \Gamma_{th})
$$

The remaining arguments are the outputs for the right and left values. `var_datar` and `var_datal` are arrays of length `num_vars`.

Other than removing the variables for reconstructing $\rho_b$ and $P$, `simple_ppm` and `simple_ppm_no_rho_P` are identical, as are the internal functions `ppm_Ur_Ul` and `ppm_Ur_Ul_no_rho_P`.

<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
!rm -f Tut*.out Tut*.aux Tut*.log